In [3]:
import os,sys

In [1]:
pwd

'd:\\Chest-Cancer-classification\\expariment'

In [4]:
os.chdir('../')


In [5]:
pwd


'd:\\Chest-Cancer-classification'

In [77]:

os.environ["MLFLOW_TRACKING_URI"]='https://dagshub.com/arpanchakraborty23/Chest-Cancer-classification.mlflow '
os.environ["MLFLOW_TRACKING_USERNAME"]="arpanchakraborty23"
os.environ["MLFLOW_TRACKING_PASSWORD"]='c0bea69b1b22e7ab1aa901fe69431974b56f94a6'

In [57]:
import tensorflow as tf

In [39]:
from tensorflow import keras
from keras.models import load_model

In [40]:
model=load_model('model\model.h5')

In [72]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    train_data:Path
    all_params: dict
    mlflow_url:str
    image_size:list
    batch_size:int


In [66]:
from src.utils.main_utils import read_yaml,create_dir,save_json
from src.constant import *

In [73]:
class ConfigManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 param_path=PARAMS_FILE_PATH) -> None:
        self.config=read_yaml(config_path)
        self.param=read_yaml(param_path)

        create_dir([self.config.artifacts_root])

    def get_eval_config(self)-> EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model='model/model.h5',
            train_data='artifacts\data_ingestion\Chest-CT-Scan-data',
            mlflow_url='https://dagshub.com/arpanchakraborty23/Chest-Cancer-classification.mlflow',
            all_params=self.param,
            image_size=self.param.IMAGE_SIZE,
            batch_size=self.param.BATCH_SIZE
        )    

        return eval_config

In [74]:
import mlflow
import mlflow.sklearn
import mlflow.keras
from urllib.parse import urlparse

In [75]:
class Evaluation:
    def __init__(self,config:EvaluationConfig) -> None:
        self.config=config

    def valid_genrator_fun(self):
           
            datagenerator_kwargs = dict(
                rescale = 1./255,
                validation_split=0.30
            )

            dataflow_kwargs = dict(
                target_size=self.config.image_size[:-1],
                batch_size=self.config.batch_size,
                interpolation="bilinear"
            )

            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.train_data,
                subset="validation",
                shuffle=False,
                **dataflow_kwargs
            )

    @staticmethod
    def load_model(path:Path):
         return load_model(path)

    def evaluation(self):
         self.model=self.load_model(self.config.path_of_model)
         self.valid_genrator_fun()
         self.score=model.evaluate(self.valid_generator)
         self.save_score()

    def save_score(self):
         scores={f'loss :{self.score[0]}, accuracy {self.save_score[1]}'}

         save_json(path=Path('score.json'),data=scores)

    def log_to_mlflow(self):
         mlflow.set_registry_uri(self.config.mlflow_url)
         track_url_type=urlparse(mlflow.get_tracking_uri()).scheme

         with mlflow.start_run():
              mlflow.log_param(self.config.all_params)
              mlflow.log_metrics(
                   f'loss :{self.score[0]}, accuracy {self.save_score[1]}'
              )
              if track_url_type !='file':
                   mlflow.keras.log_model(self.model,registered_model_name="VGG16Model")

              else:
                   mlflow.keras.load_model(self.model,'model')



In [78]:
try:
    config=ConfigManager()
    eval_config=config.get_eval_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_to_mlflow()
except Exception as e:
    raise e

Found 102 images belonging to 2 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 52s 7s/step - accuracy: 0.7966 - loss: 0.4781


TypeError: 'method' object is not subscriptable